# Predicting Price with Size, Location, and Neighborhood

In [2]:
pwd

'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model'

## Library Importing

In [3]:
from glob import glob
import pandas as pd
import seaborn as sns #for data visualisation
from category_encoders import OneHotEncoder  #encoding categorical values
from ipywidgets import Dropdown, FloatSlider, IntSlider, interact #for the interactive dashboard
from sklearn.impute import SimpleImputer #3imputting missing values
from sklearn.linear_model import LinearRegression, Ridge  # noqa F401--automaed quality control code.
from sklearn.metrics import mean_absolute_error #to evaluate the model
from sklearn.pipeline import make_pipeline  #make the pipoeline
from sklearn.utils.validation import check_is_fitted #for unit testing

## Data Wrangling

In [4]:
#Loading the dataset with a function
def wrangle (filepath):
    #Load the CSV file into file path-local function applied
    df = pd.read_csv(filepath, encoding='ISO-8859-1') #read csv to the dataframe 
    #To display 'Capital Federal' only 
    mask_ba = df["place_with_parent_names"].str.contains("Capital Federal")
    #To display 'Apartment' only 
    mask_apt = df["property_type"] == "apartment" 
    #To display price below'400,000'
    mask_price = df["price_aprox_usd"] < 400_000
    #Subset
    df = df[ mask_ba & mask_apt & mask_price]
    
    #Remove outliers by 'surface_area_m2' -----removing the 10th and 90th quantiles
    low = df["surface_covered_in_m2"].quantile(0.1)
    high = df["surface_covered_in_m2"].quantile(0.9)
    
    mask_area = df["surface_covered_in_m2"].between(low, high)
    
    df = df[mask_area]
    
    #split the lat-lon column
    df[["lat", "lon"]] = df["lat-lon"].str.split(",", expand=True).astype(float)
    df.drop(columns="lat-lon", inplace=True)
    
    #split place_with_parent_names
    df["neighborhood"] = df["place_with_parent_names"].str.split("|", expand=True)[3]
    df.drop(columns="place_with_parent_names", inplace=True)
    
    print('low = ', low, ';high = ', high)
    print('df.shape = ', df.shape)
    
    return df

In [5]:
#Collecting the files with the glob function
files = glob("C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model/buenos-aires-real-estate-*.csv")
files

['C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-1.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-2.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-3.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-4.csv',
 'C:\\Users\\admin\\Desktop\\Personal_stuff\\PROJECTS\\Proj2_Price model\\buenos-aires-real-estate-5.csv']

In [6]:
#Loading the files with list comprehension to a df 
frames = [wrangle(file) for file in files]

low =  31.0 ;high =  100.0
df.shape =  (1343, 18)
low =  31.0 ;high =  100.0
df.shape =  (1315, 18)
low =  31.0 ;high =  101.70000000000005
df.shape =  (1288, 18)
low =  30.0 ;high =  98.0
df.shape =  (1305, 18)
low =  30.0 ;high =  100.0
df.shape =  (1331, 18)


In [9]:
type(frames[0]) #indicating its a list 

pandas.core.frame.DataFrame

In [11]:
frames[0].head() #dataframe 1

Unnamed: 0 operation property_type     price currency  \
4            5      sell     apartment  129000.0      USD   
9           10      sell     apartment   87000.0      USD   
29          30      sell     apartment  118000.0      USD   
40          41      sell     apartment   57000.0      USD   
41          42      sell     apartment   90000.0      USD   

    price_aprox_local_currency  price_aprox_usd  surface_total_in_m2  \
4                    1955949.6         129000.0                 76.0   
9                    1319128.8          87000.0                 48.0   
29                   1789163.2         118000.0                  NaN   
40                    864256.8          57000.0                 42.0   
41                   1364616.0          90000.0                 57.0   

    surface_covered_in_m2  price_usd_per_m2  price_per_m2  floor  rooms  \
4                    70.0       1697.368421   1842.857143    NaN    NaN   
9                    42.0       1812.500000   2071.428571    NaN    NaN   
29                   54.0               NaN   2185.185185    NaN    2.0   
40                   42.0       1357.142857   1357.142857    5.0    2.0   
41                   50.0       1578.947368   1800.000000    NaN    3.0   

    expenses                                      properati_url        lat  \
4        NaN  http://chacarita.properati.com.ar/10qlv_venta_... -34.584651   
9        NaN  http://villa-luro.properati.com.ar/12m82_venta... -34.638979   
29       NaN  http://caballito.properati.com.ar/11wqh_venta_... -34.615847   
40     364.0  http://constitucion.properati.com.ar/k2f0_vent... -34.625222   
41     450.0  http://once.properati.com.ar/suwa_venta_depart... -34.610610   

          lon  neighborhood  
4  -58.454693     Chacarita  
9  -58.500115    Villa Luro  
29 -58.459957     Caballito  
40 -58.382382  Constitución  
41 -58.412511          Once

In [17]:
#concatenate the dataframes 
df = pd.concat(frames, ignore_index = False)
df.head()

Unnamed: 0 operation property_type     price currency  \
4            5      sell     apartment  129000.0      USD   
9           10      sell     apartment   87000.0      USD   
29          30      sell     apartment  118000.0      USD   
40          41      sell     apartment   57000.0      USD   
41          42      sell     apartment   90000.0      USD   

    price_aprox_local_currency  price_aprox_usd  surface_total_in_m2  \
4                    1955949.6         129000.0                 76.0   
9                    1319128.8          87000.0                 48.0   
29                   1789163.2         118000.0                  NaN   
40                    864256.8          57000.0                 42.0   
41                   1364616.0          90000.0                 57.0   

    surface_covered_in_m2  price_usd_per_m2  price_per_m2  floor  rooms  \
4                    70.0       1697.368421   1842.857143    NaN    NaN   
9                    42.0       1812.500000   2071.428571    NaN    NaN   
29                   54.0               NaN   2185.185185    NaN    2.0   
40                   42.0       1357.142857   1357.142857    5.0    2.0   
41                   50.0       1578.947368   1800.000000    NaN    3.0   

    expenses                                      properati_url        lat  \
4        NaN  http://chacarita.properati.com.ar/10qlv_venta_... -34.584651   
9        NaN  http://villa-luro.properati.com.ar/12m82_venta... -34.638979   
29       NaN  http://caballito.properati.com.ar/11wqh_venta_... -34.615847   
40     364.0  http://constitucion.properati.com.ar/k2f0_vent... -34.625222   
41     450.0  http://once.properati.com.ar/suwa_venta_depart... -34.610610   

          lon  neighborhood  
4  -58.454693     Chacarita  
9  -58.500115    Villa Luro  
29 -58.459957     Caballito  
40 -58.382382  Constitución  
41 -58.412511          Once